The aim here is to predict the PremiumPrice based on the provided features. We split the data using Age (stratified) to make sure the Age distribution stays similar in both the train and test sets. Then, we use the feature-engineered data to see how well the models perform.

## Imports

In [1]:
import os
import sys
parent_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parent_dir)

In [2]:
import warnings
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, StackingRegressor, VotingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

import xgboost as xgb

import mlflow

from src.utils import r2_and_adjusted_r2_score as score, root_mean_squared_error as rmse
from src.mlflow_util import setup_mlflow_experiment as setup_exp

warnings.filterwarnings('ignore')

## Read Data

In [3]:
processed_data_dir = os.path.join(parent_dir, 'data', 'processed')

In [4]:
df = pd.read_csv(os.path.join(processed_data_dir, 'insurance_feature_engineered_for_model.csv'))
df.head()

,Age,Diabetes,BloodPressureProblems,AnyTransplants,AnyChronicDiseases,Height,Weight,KnownAllergies,HistoryOfCancerInFamily,NumberOfMajorSurgeries,PremiumPrice,MajorSurgeryDone,BMI
0,45,0,0,0,0,155,57,0,0,0,25000,0,23.725286
1,60,1,0,0,0,180,73,0,0,0,29000,0,22.530864
2,36,1,1,0,0,158,59,0,0,1,23000,1,23.634033
3,52,1,1,0,1,183,93,0,0,2,28000,1,27.770313
4,38,0,0,0,1,166,88,0,0,1,23000,1,31.934969


## MLFlow Setup

In [5]:
exp_name = 'Premium Prediction - stratified by Age - Engineered Features'
exp_description = '''The goal is to predict the PremiumPrice based on the given features.
Data is splitted using the Age feature to ensure that the distribution of Age is similar in both train and test sets.
Feature engineering is done to create new features like BMI and MajorSurgeryDone
'''

experiment = setup_exp(exp_name, exp_description)

experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_id=experiment_id)


<Experiment: artifact_location='file:///Users/wasimmujawar/Projects/Insurance-Price-Predictor/mlruns/710435938879440631', creation_time=1737139253236, experiment_id='710435938879440631', last_update_time=1737139253236, lifecycle_stage='active', name='ZZZ - Premium Prediction - stratified by Age - Engineered Features', tags={'mlflow.note.content': 'The goal is to predict the PremiumPrice based on the '
                        'given features.\n'
                        'Data is splitted using the Age feature to ensure that '
                        'the distribution of Age is similar in both train and '
                        'test sets.\n'
                        'Feature engineering is done to create new features '
                        'like BMI and MajorSurgeryDone\n'}>

## Model selection

### Train Test Split

We have seen in EDA, that there is a significant difference in the premium price for different age and bmi category, we will try to split the data using both age and bmi category as stratum parameter and observe the models performance

In [6]:
bins = [0, 18.5, 24.9, 29.9, 34.9, np.inf]
labels = ['Underweight', 'Normal weight', 'Overweight', 'Obesity Class 1', 'Obesity Class 2/3']

bmi_category = pd.cut(df['BMI'], bins=bins, labels=labels, right=False)

In [7]:
age_bins = [18, 25, 40, 55, np.inf]
age_labels = ['Young Adult', 'Adult', 
              'Middle Aged Adults', 'Senior']

age_category = pd.cut(df['Age'], bins=age_bins, labels=age_labels, right=False)

First lets split the data using age category

In [8]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['PremiumPrice']), df['PremiumPrice']
                                                    ,stratify=age_category, test_size=0.2, random_state=42)

### Linear Regression

In [9]:
numeric_columns = ['Age', 'Height', 'Weight', 'BMI']
binary_columns = ['Diabetes', 'BloodPressureProblems', 'AnyTransplants', 'AnyChronicDiseases', 'KnownAllergies', 'HistoryOfCancerInFamily', 'MajorSurgeryDone', 'NumberOfMajorSurgeries']

preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), numeric_columns),
    ('passthrough', 'passthrough', binary_columns)
])

pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('regressor', LinearRegression())
])

with mlflow.start_run(run_name="Linear Regression"):
    pipeline.fit(X_train, y_train)
    
    y_train_pred = pipeline.predict(X_train)
    y_test_pred = pipeline.predict(X_test)
    
    rmse_train = rmse(y_train, y_train_pred)
    rmse_test = rmse(y_test, y_test_pred)
    
    signature = mlflow.models.infer_signature(X_train, y_train_pred)

    r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
    r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])
    
    mlflow.set_tag('Model', 'Linear Regression')
    mlflow.log_metric('Train - RMSE', rmse_train)
    mlflow.log_metric('Test - RMSE', rmse_test)
    mlflow.log_metric('Train - r2 score', r2_train)
    mlflow.log_metric('Test - r2 score', r2_test)
    mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
    mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)
    mlflow.sklearn.log_model(pipeline, 'model',  signature=signature)

### Decision Tree

In [10]:
def run_mlflow_decision_tree(max_dept: int) -> None:
    with mlflow.start_run(run_name="Decision Tree"):
        model = DecisionTreeRegressor(max_depth=max_dept)
        
        model.fit(X_train, y_train)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        rmse_train = rmse(y_train, y_train_pred)
        rmse_test = rmse(y_test, y_test_pred)
        
        signature = mlflow.models.infer_signature(X_train, y_train_pred)


        r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
        r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])
        
        mlflow.set_tag('Model', 'Decision Tree Regressor')
        
        mlflow.log_param('max_depth', max_dept)
        mlflow.log_metric('Train - RMSE', rmse_train)
        mlflow.log_metric('Test - RMSE', rmse_test)
        mlflow.log_metric('Train - r2 score', r2_train)
        mlflow.log_metric('Test - r2 score', r2_test)
        mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
        mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)

        mlflow.sklearn.log_model(model, 'model',  signature=signature)

In [11]:
for depth in range(3, 9):
    run_mlflow_decision_tree(depth)

### Random Forest Regressor

In [12]:
def run_mlflow_random_forest(n_estimators: int, max_dept: int, max_feature: str = None) -> None:
    with mlflow.start_run(run_name="Random Forest"):
        model = RandomForestRegressor(n_estimators=n_estimators, 
                                        max_depth=max_dept, 
                                        max_features=max_feature, 
                                        n_jobs=-1)
        
        model.fit(X_train, y_train)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        rmse_train = rmse(y_train, y_train_pred)
        rmse_test = rmse(y_test, y_test_pred)
        
        signature = mlflow.models.infer_signature(X_train, y_train_pred)

        r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
        r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])
        
        mlflow.set_tag('Model', 'Random Forest Regressor')
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_dept)
        mlflow.log_param('max_features', max_feature)
        mlflow.log_metric('Train - RMSE', rmse_train)
        mlflow.log_metric('Test - RMSE', rmse_test)
        mlflow.log_metric('Train - r2 score', r2_train)
        mlflow.log_metric('Test - r2 score', r2_test)
        mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
        mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)
        mlflow.sklearn.log_model(model, 'model',  signature=signature)

In [13]:
n_estimator = [80, 90, 100, 110, 120]
max_depth = [3, 4, 5, 6, 7, 8]
max_features = [None, 'sqrt', 'log2']

for n_est in n_estimator:
    for depth in max_depth:
        for feature in max_features:
            run_mlflow_random_forest(n_est, depth, feature)

### Gradient Boosting Regressor

In [14]:
def run_mlflow_gbdt(loss: str, learning_rate: float, n_estimators: int, max_dept: int) -> None:
    with mlflow.start_run(run_name="GBDT"):
        model = GradientBoostingRegressor(n_estimators=n_estimators, 
                                            max_depth=max_dept, 
                                            learning_rate=learning_rate, 
                                            loss=loss)
        
        model.fit(X_train, y_train)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        rmse_train = rmse(y_train, y_train_pred)
        rmse_test = rmse(y_test, y_test_pred)
        
        signature = mlflow.models.infer_signature(X_train, y_train_pred)

        r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
        r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])
        
        mlflow.set_tag('Model', 'GBDT Regressor')
        mlflow.log_param('loss', loss)
        mlflow.log_param('learning_rate', learning_rate)
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('max_depth', max_dept)
        mlflow.log_metric('Train - RMSE', rmse_train)
        mlflow.log_metric('Test - RMSE', rmse_test)
        mlflow.log_metric('Train - r2 score', r2_train)
        mlflow.log_metric('Test - r2 score', r2_test)
        mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
        mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)
        mlflow.sklearn.log_model(model, 'model',  signature=signature)

In [15]:
loss = ['squared_error', 'huber']
n_estimator = [80, 90, 100, 110, 120]
max_depth = [3, 4, 5, 6]
learning_rate = [0.01, 0.1, 0.2, 0.3]

In [16]:
for l in loss:
    for n_est in n_estimator:
        for depth in max_depth:
            for lr in learning_rate:
                run_mlflow_gbdt(l, lr, n_est, depth)

### XGBoost Regression

In [17]:
def run_mlflow_xgboost(max_dept: int, n_estimators: int, learning_rate: float, reg_alpha: float, reg_lambda: float, child: int, booster: str) -> None:
    with mlflow.start_run(run_name="XGBoost"):
        model = xgb.XGBRegressor(max_depth=max_dept, n_estimators=n_estimators, 
                                 learning_rate=learning_rate, n_jobs=-1, reg_alpha=reg_alpha, 
                                 reg_lambda=reg_lambda, subsample=0.6, colsample_bytree=0.7, min_child_weight=child, booster=booster)
        
        model.fit(X_train, y_train)
        
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        rmse_train = rmse(y_train, y_train_pred)
        rmse_test = rmse(y_test, y_test_pred)
        
        signature = mlflow.models.infer_signature(X_train, y_train_pred)

        r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
        r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])
        
        mlflow.set_tag('Model', 'XGBoost Regressor')
        mlflow.log_param('max_depth', max_dept)
        mlflow.log_param('n_estimators', n_estimators)
        mlflow.log_param('learning_rate', learning_rate)
        mlflow.log_param('reg_alpha', reg_alpha)
        mlflow.log_param('reg_lambda', reg_lambda)
        mlflow.log_param('min_child_weight', child)
        mlflow.log_param('booster', booster)
        
        mlflow.log_metric('Train - RMSE', rmse_train)
        mlflow.log_metric('Test - RMSE', rmse_test)
        mlflow.log_metric('Train - r2 score', r2_train)
        mlflow.log_metric('Test - r2 score', r2_test)
        mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
        mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)
        mlflow.sklearn.log_model(model, 'model',  signature=signature)

In [18]:
n_estimator = [80, 90, 100]
reg_alpha = [0.3, 0.5, 0.7]
reg_lambda = [0.3, 0.5, 0.7]
max_depth = [5, 6, 7]
learning_rate = [0.08, 0.12] #[0.02, 0.03, 0.04]
min_child_weights = [2, 3]
boosters = ['gbtree', 'dart']

for n_est in n_estimator:
    for depth in max_depth:
        for lr in learning_rate:
            for alpha in reg_alpha:
                for lam in reg_lambda:
                    for child in min_child_weights:
                        for booster in boosters:
                            run_mlflow_xgboost(depth, n_est, lr, alpha, lam, child, booster)

### Stacking Regression

In [19]:
base_learner = [
    ('gb1', GradientBoostingRegressor(n_estimators=110, max_depth=5, learning_rate=0.1, loss='huber')),
    ('gb2', GradientBoostingRegressor(n_estimators=120, max_depth=5, learning_rate=0.1, loss='huber')),
    ('rd1', RandomForestRegressor(n_estimators=100, max_depth=7, n_jobs=-1)),
    ('rd2', RandomForestRegressor(n_estimators=90, max_depth=7, n_jobs=-1)),
    ('xg1', xgb.XGBRegressor(max_depth=6, n_estimators=90, learning_rate=0.03, reg_alpha=0.5, reg_lambda=0.7)),
    ('xg2', xgb.XGBRegressor(max_depth=6, n_estimators=80, learning_rate=0.04, reg_alpha=0.1, reg_lambda=0.7))
]

final_estimator = LinearRegression()

with mlflow.start_run(run_name="Stacking Regressor"):
    model = StackingRegressor(estimators=base_learner, final_estimator=final_estimator)
    
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    rmse_train = rmse(y_train, y_train_pred)
    rmse_test = rmse(y_test, y_test_pred)

    signature = mlflow.models.infer_signature(X_train, y_train_pred)

    r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
    r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])

    mlflow.set_tag('Model', 'Stacking Regressor')
    mlflow.log_metric('Train - RMSE', rmse_train)
    mlflow.log_metric('Test - RMSE', rmse_test)
    mlflow.log_metric('Train - r2 score', r2_train)
    mlflow.log_metric('Test - r2 score', r2_test)
    mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
    mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)
    mlflow.sklearn.log_model(model, 'model',  signature=signature)

### Voting Regressor

In [20]:
with mlflow.start_run(run_name='Voting Regressor'):
    model = VotingRegressor(estimators=base_learner)
    
    model.fit(X_train, y_train)

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    rmse_train = rmse(y_train, y_train_pred)
    rmse_test = rmse(y_test, y_test_pred)

    signature = mlflow.models.infer_signature(X_train, y_train_pred)

    r2_train, adj_r2_train = score(y_train, y_train_pred, X_train.shape[0], X_train.shape[1])
    r2_test, adj_r2_test = score(y_test, y_test_pred, X_test.shape[0], X_test.shape[1])

    mlflow.set_tag('Model', 'Voting Regressor')
    mlflow.log_metric('Train - RMSE', rmse_train)
    mlflow.log_metric('Test - RMSE', rmse_test)
    mlflow.log_metric('Train - r2 score', r2_train)
    mlflow.log_metric('Test - r2 score', r2_test)
    mlflow.log_metric('Train - Adjusted r2 score', adj_r2_train)
    mlflow.log_metric('Test - Adjusted r2 score', adj_r2_test)
    mlflow.sklearn.log_model(model, 'model',  signature=signature)